In [192]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import IsolationForest

data = pd.read_csv("../../datasets/normal_dataset.csv")


In [193]:
print(data.shape)
print(data.head())

(99381, 21)
             Time  NodeID          MAC  SF           CF  TX      BW  CR  \
0  8/30/2025 0:00     197  00-00-00-C5  11  868300000.0  14  125000   1   
1  8/30/2025 0:00     239  00-00-00-EF  12  868100000.0  14  125000   1   
2  8/30/2025 0:00      22  00-00-00-16   9  868500000.0  14  125000   1   
3  8/30/2025 0:00     103  00-00-00-67   7  868300000.0  14  125000   1   
4  8/30/2025 0:00      91  00-00-00-5B   7  868500000.0  14  125000   1   

        SNR     RSSI  ...    payload payloadSize  \
0   3.35339 -123.147  ...  DataFrame           8   
1  -1.76860 -128.269  ...  DataFrame           8   
2   2.34693 -124.153  ...  DataFrame           8   
3  17.10600 -109.394  ...  DataFrame           8   
4  13.04460 -113.455  ...  DataFrame           8   

   numReceivedPerNode[nodeNumber-1]  PDRPerNode  \
0                                 1         1.0   
1                                 1         1.0   
2                                 1         1.0   
3                   